# Cleaning

**We have cleaned the following in the given dataset membership.csv :**
- Invalid dates
- Dropping duplicates
- Null or missing values
- Check for 0 or negative price values

**We have done the following for improved readability :**
- Rename Clients
- Rename ContactIds

**Insights**
- A person (ContactId) can be a member of two different products at the same time.

In [173]:
# Imports
import pandas as pd

# ImpexiumInsights is the membership data
df = pd.read_csv('membership.csv')
df.head(5)

# Change clients to a more readable format as the following:
# Client_1 -> ICSC
# Client_2 -> SIOR
# Client_3 -> NAIOP
df['Client'] = df['Client'].replace('Client_1', 'ICSC')
df['Client'] = df['Client'].replace('Client_2', 'SIOR')
df['Client'] = df['Client'].replace('Client_3', 'NAIOP')
df.head(10)

,ContactId,ClientId,Client,ProductId,ProductType,EffectiveDate,ExpireDate,Total
0,0x566B63D73878FD95925F7B6716FAB095,0xCAB892266CB7872B20CED441410BFF5E,SIOR,0x9E8B160226C9FE22A910C782CE5076E2,Membership,9/1/21,8/31/22,104.49
1,0x77D707DC7112E8379C27E3A1AF0EC034,0xCAB892266CB7872B20CED441410BFF5E,SIOR,0x9E8B160226C9FE22A910C782CE5076E2,Membership,8/1/21,7/31/22,104.49
2,0xC36639462513FE7C2413AF7449B281BD,0xCAB892266CB7872B20CED441410BFF5E,SIOR,0xCF82E756F655A4E1518567DCDB80CCBF,Membership,6/27/21,6/26/22,36.45
3,0xC36639462513FE7C2413AF7449B281BD,0xCAB892266CB7872B20CED441410BFF5E,SIOR,0x9E8B160226C9FE22A910C782CE5076E2,Membership,6/19/21,6/18/22,104.49
4,0xBDF36B8927A18BC973A2AE91E7F5112F,0xCAB892266CB7872B20CED441410BFF5E,SIOR,0x9E8B160226C9FE22A910C782CE5076E2,Membership,6/27/21,6/26/22,104.49
5,0x18CC10100F771A69BFD13098384C7E46,0xCAB892266CB7872B20CED441410BFF5E,SIOR,0x9E8B160226C9FE22A910C782CE5076E2,Membership,8/2/21,8/1/22,104.49
6,0x0E49629AB8D42E50157CFDE15186FD8B,0xCAB892266CB7872B20CED441410BFF5E,SIOR,0x9E8B160226C9FE22A910C782CE5076E2,Membership,6/29/21,6/28/22,104.49
7,0xB584C9347DE96FFE23C5BD103626F03F,0xCAB892266CB7872B20CED441410BFF5E,SIOR,0x9E8B160226C9FE22A910C782CE5076E2,Membership,6/28/21,6/27/22,104.49
8,0x7181625DB00B71EAA5FEB7C1632F92EB,0xCAB892266CB7872B20CED441410BFF5E,SIOR,0x9E8B160226C9FE22A910C782CE5076E2,Membership,6/28/21,6/27/22,104.49
9,0xD35391A8B2400414D05B610DBC052102,0xCAB892266CB7872B20CED441410BFF5E,SIOR,0x9E8B160226C9FE22A910C782CE5076E2,Membership,1/1/20,12/31/20,104.49


In [174]:
# Removing all entries where EffectiveDate > ExpireDate
df['EffectiveDate'] = pd.to_datetime(df['EffectiveDate'], format='%m/%d/%y')
df['ExpireDate'] = pd.to_datetime(df['ExpireDate'], format='%m/%d/%y')
valid_df = df[df['EffectiveDate'] <= df['ExpireDate']]

# Check 
invalid_df = valid_df[valid_df['EffectiveDate'] > valid_df['ExpireDate']]
print("Are there any entries in the new data frame that are invalid for dates?", not(invalid_df.to_xarray))


# Check passed so modify data frame
df = valid_df

Are there any entries in the new data frame that are invalid for dates? False


In [175]:
# Are there any prices that are 0 or negative?
invalid_rows = df[df['Total'] <= 0]

# Check
print("Are there any prices that are 0 or negative?", not(invalid_rows.to_xarray))

Are there any prices that are 0 or negative? False


In [176]:
# Can different people be members of the same ProductId at the same time?
unique_products = df.groupby('ContactId')['ProductId'].nunique()
subscriptions = unique_products[unique_products > 1]

# Check
print("Can different people be members of the same ProductId at the same time?", not(not({subscriptions.to_dict})))

Can different people be members of the same ProductId at the same time? True


In [177]:
# Dropping all duplicates that have the same ContactId, ProductId, EffectiveDate, and ExpireDate
df['EffectiveDate'] = pd.to_datetime(df['EffectiveDate'], format='%m/%d/%y')
df['ExpireDate'] = pd.to_datetime(df['ExpireDate'], format='%m/%d/%y')
df_unique = df.drop_duplicates(subset=['ContactId', 'ProductId', 'EffectiveDate', 'ExpireDate'])

# Check 
duplicates_check = df_unique.duplicated(subset=['ContactId', 'ProductId', 'EffectiveDate', 'ExpireDate'], keep=False)
no_duplicates = not duplicates_check.any()
print("There are no duplicates in the new data frame, based on the specified columns above?", no_duplicates)

# Check passed so modify data frame
df = df_unique

There are no duplicates in the new data frame, based on the specified columns above? True


In [178]:
# Identifying any null or missing values
missing_entries = df[df.isnull().any(axis=1) | df.applymap(lambda x: x == '').any(axis=1)]

# Check
print("Are there any null or missing values in the data frame?", not(missing_entries.to_xarray))

Are there any null or missing values in the data frame? False


In [179]:
# Generate a unique "Member X" identifier for each unique ContactId
temp_df = df.copy()
unique_ids = temp_df['ContactId'].unique()
member_mapping = {id_: f"Member {i+1}" for i, id_ in enumerate(unique_ids)}
temp_df['ContactId'] = temp_df['ContactId'].map(member_mapping)

# Check
print(temp_df.head(5))

# Check passed so modify data frame
df = temp_df

  ContactId                            ClientId Client  \
0  Member 1  0xCAB892266CB7872B20CED441410BFF5E   SIOR   
1  Member 2  0xCAB892266CB7872B20CED441410BFF5E   SIOR   
2  Member 3  0xCAB892266CB7872B20CED441410BFF5E   SIOR   
3  Member 3  0xCAB892266CB7872B20CED441410BFF5E   SIOR   
4  Member 4  0xCAB892266CB7872B20CED441410BFF5E   SIOR   

                            ProductId ProductType EffectiveDate ExpireDate  \
0  0x9E8B160226C9FE22A910C782CE5076E2  Membership    2021-09-01 2022-08-31   
1  0x9E8B160226C9FE22A910C782CE5076E2  Membership    2021-08-01 2022-07-31   
2  0xCF82E756F655A4E1518567DCDB80CCBF  Membership    2021-06-27 2022-06-26   
3  0x9E8B160226C9FE22A910C782CE5076E2  Membership    2021-06-19 2022-06-18   
4  0x9E8B160226C9FE22A910C782CE5076E2  Membership    2021-06-27 2022-06-26   

    Total  
0  104.49  
1  104.49  
2   36.45  
3  104.49  
4  104.49  


In [180]:
# Saving updated data frame to the csv
# Always run this at the end of your work session:
df.to_csv('modified_membership.csv', index=False)

In [181]:
# Membership Ranges
# Come back to this after cleaning
# def assign_membership(total):
#     if total < 350:
#         return 'Standard'
#     elif 350 <= total < 700:
#         return 'Silver'
#     elif total >= 700:
#         return 'Gold'

# product_totals = df.copy()
# product_totals['Membership'] = product_totals['Total'].apply(assign_membership)
# product_id_to_membership = product_totals.set_index('ProductId')['Membership'].to_dict()


# df['ProductId'] = df['ProductId'].map(product_id_to_membership)

# Analysis

**Questions we looked into :**
- What is the proportion of people who renew? 
- How many times do unique contact ID’s renew their memberships?

In [182]:
# Sort the DataFrame
df_sorted = df.sort_values(by=['ContactId', 'ProductId', 'EffectiveDate', 'ExpireDate'])
df_sorted['EffectiveDate'] = pd.to_datetime(df_sorted['EffectiveDate'], format='%m/%d/%Y')
df_sorted['ExpireDate'] = pd.to_datetime(df_sorted['ExpireDate'], format='%m/%d/%Y')

# Custom function to identify renewals
def check_continuing_membership(group):
    group['Renewed'] = group['EffectiveDate'].shift(-1) > group['ExpireDate']
    return group

df_renewals = df_sorted.groupby(['ContactId', 'ProductId']).apply(check_continuing_membership)
df_renewed_memberships = df_renewals[df_renewals['Renewed'] == True]
renewed_contact_ids = df_renewed_memberships['ContactId'].unique()
total_unique_contact_ids = df['ContactId'].nunique()

# Check
print(len(renewed_contact_ids))
print(total_unique_contact_ids)

/var/folders/ss/yqcdmh2s71390cvwl12h39300000gn/T/ipykernel_73664/2462553618.py:11: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_renewals = df_sorted.groupby(['ContactId', 'ProductId']).apply(check_continuing_membership)


40621
94336
